In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
#载入数据库
mnist = input_data.read_data_sets("/root/code/MNIST_data/",one_hot= True)


# n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x,[-1,28,28,1])

W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 14*14*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)

#此时连接推测是rnn维度需要二维 此时 h_pool2




# 第一个全连接层
W_fc1 = weight_variable([7*7*64,784])
b_fc1 = bias_vairable([784])
# 7*7*64的图像变成1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# print("咋回事怎么变",h_fc1)

# # 第二个全连接层
# W_fc2 = weight_variable([1024,10])
# b_fc2 = bias_vairable([10])
# logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2












#初始化权重
weights = tf.Variable(tf.truncated_normal([100,10],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[10]))

def RUN(X,weights,biases):
    # inputs = [batch_size,max_time,n_inputs]
#     print('2',X)
    inputs = tf.reshape(X,[-1,28,28])
    #定义LSTM基本CELL
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32,time_major = False)
#     print('3',final_state[1])
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results

#计算RUN的返回结果
prediction = RUN(h_fc1,weights,biases)
# print(3333)
# print(prediction)
# print(555555555)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver = tf.train.Saver()
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start = datetime.now()
    for epoch in range(5001):

        batch_xs,batch_ys =mnist.train.next_batch(50)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))

    end = datetime.now()
    latency = (end - start).total_seconds()
    print('训练5000步总时长为 '+str(latency)+' 秒 ')
    saver.save(sess,"/root/tf_nodel/a_model.ckpt")




# # 第一个全连接层
# W_fc1 = weight_variable([7*7*64,1024])
# b_fc1 = bias_vairable([1024])
# # 7*7*64的图像变成1维向量
# h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# # 第二个全连接层
# W_fc2 = weight_variable([1024,10])
# b_fc2 = bias_vairable([10])
# logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
# prediction = tf.nn.sigmoid(logits)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# prediction_2 = tf.nn.softmax(prediction)
# correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for epoch in range(21):
#         for batch in range(n_batch):
#             batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#             sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
#         acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
#         print("Iter: " + str(epoch) + ", acc: " + str(acc))

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /root/code/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /root/code/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Pleas

Iter141,testing accuracy=0.6673
Iter142,testing accuracy=0.6702
Iter143,testing accuracy=0.6683
Iter144,testing accuracy=0.6693
Iter145,testing accuracy=0.6688
Iter146,testing accuracy=0.6697
Iter147,testing accuracy=0.6693
Iter148,testing accuracy=0.6694
Iter149,testing accuracy=0.6695
Iter150,testing accuracy=0.6697
Iter151,testing accuracy=0.6691
Iter152,testing accuracy=0.6696
Iter153,testing accuracy=0.6702
Iter154,testing accuracy=0.672
Iter155,testing accuracy=0.6735
Iter156,testing accuracy=0.6741
Iter157,testing accuracy=0.6735
Iter158,testing accuracy=0.6716
Iter159,testing accuracy=0.6709
Iter160,testing accuracy=0.6707
Iter161,testing accuracy=0.6705
Iter162,testing accuracy=0.6687
Iter163,testing accuracy=0.6672
Iter164,testing accuracy=0.6689
Iter165,testing accuracy=0.6706
Iter166,testing accuracy=0.6711
Iter167,testing accuracy=0.6712
Iter168,testing accuracy=0.6714
Iter169,testing accuracy=0.6722
Iter170,testing accuracy=0.6741
Iter171,testing accuracy=0.6729
Iter172,t

Iter398,testing accuracy=0.9127
Iter399,testing accuracy=0.9076
Iter400,testing accuracy=0.9017
Iter401,testing accuracy=0.9024
Iter402,testing accuracy=0.9074
Iter403,testing accuracy=0.917
Iter404,testing accuracy=0.9205
Iter405,testing accuracy=0.9202
Iter406,testing accuracy=0.9153
Iter407,testing accuracy=0.9086
Iter408,testing accuracy=0.9049
Iter409,testing accuracy=0.909
Iter410,testing accuracy=0.9132
Iter411,testing accuracy=0.9229
Iter412,testing accuracy=0.9253
Iter413,testing accuracy=0.9214
Iter414,testing accuracy=0.9172
Iter415,testing accuracy=0.908
Iter416,testing accuracy=0.9016
Iter417,testing accuracy=0.8953
Iter418,testing accuracy=0.8989
Iter419,testing accuracy=0.906
Iter420,testing accuracy=0.9152
Iter421,testing accuracy=0.9174
Iter422,testing accuracy=0.9099
Iter423,testing accuracy=0.9056
Iter424,testing accuracy=0.9069
Iter425,testing accuracy=0.9095
Iter426,testing accuracy=0.918
Iter427,testing accuracy=0.9208
Iter428,testing accuracy=0.9231
Iter429,testi

Iter655,testing accuracy=0.9391
Iter656,testing accuracy=0.9392
Iter657,testing accuracy=0.9395
Iter658,testing accuracy=0.9407
Iter659,testing accuracy=0.9427
Iter660,testing accuracy=0.9448
Iter661,testing accuracy=0.9463
Iter662,testing accuracy=0.9458
Iter663,testing accuracy=0.9449
Iter664,testing accuracy=0.9412
Iter665,testing accuracy=0.9384
Iter666,testing accuracy=0.9359
Iter667,testing accuracy=0.9346
Iter668,testing accuracy=0.9356
Iter669,testing accuracy=0.9392
Iter670,testing accuracy=0.942
Iter671,testing accuracy=0.9421
Iter672,testing accuracy=0.9412
Iter673,testing accuracy=0.9402
Iter674,testing accuracy=0.939
Iter675,testing accuracy=0.9384
Iter676,testing accuracy=0.9395
Iter677,testing accuracy=0.9403
Iter678,testing accuracy=0.9384
Iter679,testing accuracy=0.9353
Iter680,testing accuracy=0.9336
Iter681,testing accuracy=0.933
Iter682,testing accuracy=0.9308
Iter683,testing accuracy=0.9304
Iter684,testing accuracy=0.9338
Iter685,testing accuracy=0.9382
Iter686,tes

...

Iter3912,testing accuracy=0.976
Iter3913,testing accuracy=0.9753
Iter3914,testing accuracy=0.9743
Iter3915,testing accuracy=0.9741
Iter3916,testing accuracy=0.9729
Iter3917,testing accuracy=0.9722
Iter3918,testing accuracy=0.9729
Iter3919,testing accuracy=0.9737
Iter3920,testing accuracy=0.9746
Iter3921,testing accuracy=0.9747
Iter3922,testing accuracy=0.9754
Iter3923,testing accuracy=0.9764
Iter3924,testing accuracy=0.9777
Iter3925,testing accuracy=0.9782
Iter3926,testing accuracy=0.9773
Iter3927,testing accuracy=0.9763
Iter3928,testing accuracy=0.9755
Iter3929,testing accuracy=0.9743
Iter3930,testing accuracy=0.9731
Iter3931,testing accuracy=0.973
Iter3932,testing accuracy=0.9735
Iter3933,testing accuracy=0.9741
Iter3934,testing accuracy=0.9731
Iter3935,testing accuracy=0.9731
Iter3936,testing accuracy=0.9734
Iter3937,testing accuracy=0.9732
Iter3938,testing accuracy=0.9736
Iter3939,testing accuracy=0.9742
Iter3940,testing accuracy=0.9755
Iter3941,testing accuracy=0.9767
Iter3942,tes

Iter4162,testing accuracy=0.979
Iter4163,testing accuracy=0.9782
Iter4164,testing accuracy=0.9774
Iter4165,testing accuracy=0.9785
Iter4166,testing accuracy=0.9781
Iter4167,testing accuracy=0.9782
Iter4168,testing accuracy=0.978
Iter4169,testing accuracy=0.9787
Iter4170,testing accuracy=0.9796
Iter4171,testing accuracy=0.9788
Iter4172,testing accuracy=0.9785
Iter4173,testing accuracy=0.9777
Iter4174,testing accuracy=0.977
Iter4175,testing accuracy=0.976
Iter4176,testing accuracy=0.9756
Iter4177,testing accuracy=0.9739
Iter4178,testing accuracy=0.9724
Iter4179,testing accuracy=0.9712
Iter4180,testing accuracy=0.9705
Iter4181,testing accuracy=0.9702
Iter4182,testing accuracy=0.9701
Iter4183,testing accuracy=0.97
Iter4184,testing accuracy=0.9702
Iter4185,testing accuracy=0.9711
Iter4186,testing accuracy=0.9719
Iter4187,testing accuracy=0.9728
Iter4188,testing accuracy=0.973
Iter4189,testing accuracy=0.9738
Iter4190,testing accuracy=0.9746
Iter4191,testing accuracy=0.9752
Iter4192,testing 

Iter4412,testing accuracy=0.9781
Iter4413,testing accuracy=0.9774
Iter4414,testing accuracy=0.9775
Iter4415,testing accuracy=0.9767
Iter4416,testing accuracy=0.9761
Iter4417,testing accuracy=0.9754
Iter4418,testing accuracy=0.9744
Iter4419,testing accuracy=0.9735
Iter4420,testing accuracy=0.972
Iter4421,testing accuracy=0.9706
Iter4422,testing accuracy=0.9704
Iter4423,testing accuracy=0.9705
Iter4424,testing accuracy=0.9738
Iter4425,testing accuracy=0.9752
Iter4426,testing accuracy=0.9765
Iter4427,testing accuracy=0.9779
Iter4428,testing accuracy=0.9785
Iter4429,testing accuracy=0.9787
Iter4430,testing accuracy=0.9787
Iter4431,testing accuracy=0.9786
Iter4432,testing accuracy=0.979
Iter4433,testing accuracy=0.9776
Iter4434,testing accuracy=0.9769
Iter4435,testing accuracy=0.9761
Iter4436,testing accuracy=0.9754
Iter4437,testing accuracy=0.9756
Iter4438,testing accuracy=0.9753
Iter4439,testing accuracy=0.9756
Iter4440,testing accuracy=0.9762
Iter4441,testing accuracy=0.9761
Iter4442,tes

Iter4662,testing accuracy=0.9776
Iter4663,testing accuracy=0.9784
Iter4664,testing accuracy=0.9774
Iter4665,testing accuracy=0.9767
Iter4666,testing accuracy=0.9772
Iter4667,testing accuracy=0.9759
Iter4668,testing accuracy=0.9747
Iter4669,testing accuracy=0.9728
Iter4670,testing accuracy=0.9707
Iter4671,testing accuracy=0.9691
Iter4672,testing accuracy=0.9689
Iter4673,testing accuracy=0.9678
Iter4674,testing accuracy=0.9679
Iter4675,testing accuracy=0.9679
Iter4676,testing accuracy=0.9681
Iter4677,testing accuracy=0.9717
Iter4678,testing accuracy=0.976
Iter4679,testing accuracy=0.9776
Iter4680,testing accuracy=0.9788
Iter4681,testing accuracy=0.9791
Iter4682,testing accuracy=0.9783
Iter4683,testing accuracy=0.9778
Iter4684,testing accuracy=0.9779
Iter4685,testing accuracy=0.9779
Iter4686,testing accuracy=0.9771
Iter4687,testing accuracy=0.9773
Iter4688,testing accuracy=0.9772
Iter4689,testing accuracy=0.977
Iter4690,testing accuracy=0.9764
Iter4691,testing accuracy=0.976
Iter4692,test

Iter4912,testing accuracy=0.9775
Iter4913,testing accuracy=0.9754
Iter4914,testing accuracy=0.9728
Iter4915,testing accuracy=0.972
Iter4916,testing accuracy=0.9708
Iter4917,testing accuracy=0.9713
Iter4918,testing accuracy=0.9736
Iter4919,testing accuracy=0.9752
Iter4920,testing accuracy=0.9778
Iter4921,testing accuracy=0.9787
Iter4922,testing accuracy=0.9796
Iter4923,testing accuracy=0.9809
Iter4924,testing accuracy=0.9817
Iter4925,testing accuracy=0.9817
Iter4926,testing accuracy=0.9818
Iter4927,testing accuracy=0.9819
Iter4928,testing accuracy=0.9816
Iter4929,testing accuracy=0.9812
Iter4930,testing accuracy=0.9806
Iter4931,testing accuracy=0.9795
Iter4932,testing accuracy=0.9786
Iter4933,testing accuracy=0.9771
Iter4934,testing accuracy=0.9764
Iter4935,testing accuracy=0.9768
Iter4936,testing accuracy=0.9761
Iter4937,testing accuracy=0.9758
Iter4938,testing accuracy=0.9754
Iter4939,testing accuracy=0.9749
Iter4940,testing accuracy=0.9753
Iter4941,testing accuracy=0.9761
Iter4942,te

In [ ]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data

#载入数据
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#载入图片是 28*28
n_inputs = 28 #输入一共28行
max_time = 28 #一共28行
lstm_size = 100#隐层单元 block
n_classes = 10 #10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples//batch_size #计算一共有多少批次

#这里的none 表示第一维度可以为任意长度
x = tf.placeholder(tf.float32,[None,784])
#正确的标签
y = tf.placeholder(tf.float32,[None,10])

#初始化权重
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义run网络
def RUN(X,weights,biases):
    # inputs = [batch_size,max_time,n_inputs]
    print('2',X)
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    print('3',final_state[1])
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results

#计算RUN的返回结果
prediction = RUN(x,weights,biases)
#损失函数
print(111111)
print(prediction)
print(555555555)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys =mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))





InvalidArgumentError: logits and labels must be broadcastable: logits_size=[1600,10] labels_size=[50,10]
	 [[{{node softmax_cross_entropy_with_logits_sg}}]]
     
可看出 报错为类型不符不能运算
不明白 logits_size是什么决定的？？ 知道了labels_size是批次和标签总数决定的



Tensor("Softmax:0", shape=(?, 28, 10), dtype=float32)  需要shape=(?, 10)




